In [43]:
import warnings

warnings.filterwarnings(
    "ignore",
    message="torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.",
)

warnings.filterwarnings(
    "ignore",
    message="torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.",
)

warnings.filterwarnings(
    "ignore",
    message="This function should be used just for training as it may exhibit reduced inference performance. For inference, please use forward_flashattn_inference.",
)

In [3]:
# Written by Yukang Chen
# Some code based on https://github.com/epfml/landmark-attention
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import sys
import io
import os
import copy
import json
import math
import logging
import pandas as pd
from dataclasses import dataclass, field
from typing import Dict, Optional, Sequence

import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer, DataCollatorForLanguageModeling
from llama_attn_replace_sft import replace_llama_attn
from gptneox_attn_replace import replace_gpt_neox_attn
from peft import LoraConfig, get_peft_model
from torch.distributed import barrier

IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"

def _make_r_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f = open(f, mode=mode)
    return f

def jload(f, mode="r"):
    """Load a .json file into a dictionary."""
    f = _make_r_io_base(f, mode)
    jdict = json.load(f)
    f.close()
    return jdict

PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
    "prompt_no_input_llama2":(
        "[INST] <<SYS>>\n"
        "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\n"
        "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n"
        "<</SYS>> \n\n {instruction} [/INST]"
    ),
    "prompt_input_llama2": (
        "[INST] <<SYS>>\n"
        "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\n"
        "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n"
        "<</SYS>> \n\n {instruction} \n{input} [/INST]"
    ),
    "prompt_llama2": "[INST]{instruction}[/INST]",
    "prompt_input_diploma_special":(
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\nBelow is a diploma text. Your task is to generate abstract of this diploma.\n\n### Input:\n{input}\n\n### Response:"
    ),
}


@dataclass
class ModelArguments:
    model_name_or_path: Optional[str] = field(default="EleutherAI/pythia-1.4b-deduped")
    model_type: Optional[str] = field(default="llama")


@dataclass
class DataArguments:
    train_data_path: str = field(default=None, metadata={"help": "Path to the training data."})
    val_data_path: str = field(default=None, metadata={"help": "Path to the validation data."})
    nrows: int = 1
    diploma_prefix_len: int = 1


@dataclass
class TrainingArguments(transformers.TrainingArguments):
    cache_dir: Optional[str] = field(default=None)
    optim: str = field(default="adamw_torch")
    model_max_length: int = field(
        default=8192 * 4,
        metadata={"help": "Maximum sequence length. Sequences will be right padded (and possibly truncated)."},
    )
    use_flash_attn: bool = field(
        default=True,
        metadata={"help": "Whether use flash attention for training."},
    )
    use_full_attn: bool = field(
        default=False,
        metadata={"help": "Whether to use plain, full-attention for training."},
    )
    low_rank_training: bool = field(
        default=True,
        metadata={"help": "Whether use low rank adaptation for training."},
    )
    trainable_params: str = field(
        default="embed,norm",
        metadata={"help": "Additional trainable parameters except LoRA weights, if low rank training."},
    )

def smart_tokenizer_and_embedding_resize(
    special_tokens_dict: Dict,
    tokenizer: transformers.PreTrainedTokenizer,
    model: transformers.PreTrainedModel,
):
    """Resize tokenizer and embedding.

    Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
    """
    num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer))

    if num_new_tokens > 0:
        input_embeddings = model.get_input_embeddings().weight.data
        output_embeddings = model.get_output_embeddings().weight.data

        input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)
        output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)

        input_embeddings[-num_new_tokens:] = input_embeddings_avg
        output_embeddings[-num_new_tokens:] = output_embeddings_avg


def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )


def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)


class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer, nrows: int, diploma_prefix_len: int):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        data_table = pd.read_csv(data_path)
        data_table = data_table.sample(min(len(data_table), nrows))

        logging.warning("Formatting inputs...")

        prompt_input_diploma = PROMPT_DICT["prompt_input_diploma_special"]
        sources = [
            prompt_input_diploma.format(input=diploma[:diploma_prefix_len])
            for diploma in data_table["diploma"]
        ]

        targets = [f"{abstract}{tokenizer.eos_token}" for abstract in data_table["abstract"]]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )


def make_supervised_data_module(tokenizer: transformers.PreTrainedTokenizer, data_args) -> Dict:
    """Make dataset and collator for supervised fine-tuning."""
    train_dataset = SupervisedDataset(tokenizer=tokenizer, data_path=data_args.train_data_path, nrows=data_args.nrows, diploma_prefix_len=data_args.diploma_prefix_len)
    val_dataset = SupervisedDataset(tokenizer=tokenizer, data_path=data_args.val_data_path, nrows=data_args.nrows, diploma_prefix_len=data_args.diploma_prefix_len)
    data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
    return dict(train_dataset=train_dataset, eval_dataset=val_dataset, data_collator=data_collator)


def train(model_args, data_args, training_args):
    print("Begin train")
    
    print("Parsed arguments")

    # NOTE: May expand supported model types in the future
    if model_args.model_type == "gpt-neox":
        replace_gpt_neox_attn(training_args.use_flash_attn, training_args.use_full_attn)
    else:
        replace_llama_attn(training_args.use_flash_attn, training_args.use_full_attn)

    # Set RoPE scaling factor
    config = transformers.AutoConfig.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=training_args.cache_dir,
    )

    orig_rope_scaling = getattr(config, "rope_scaling", None)
    if orig_rope_scaling is None:
        orig_rope_scaling = {"factor": 1}
    orig_rope_scaling_factor = orig_rope_scaling["factor"] if "factor" in orig_rope_scaling.keys() else 1
    orig_ctx_len = getattr(config, "max_position_embeddings", None)
    if orig_ctx_len:
        orig_ctx_len *= orig_rope_scaling_factor
        if training_args.model_max_length > orig_ctx_len:
            scaling_factor = float(math.ceil(training_args.model_max_length / orig_ctx_len))
            config.rope_scaling = {"type": "linear", "factor": scaling_factor}
            
    print("Created config")

    # Load model and tokenizer
    model = transformers.AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
        config=config,
        cache_dir=training_args.cache_dir,
        torch_dtype=torch.bfloat16,
    )
    
    print("Loaded model")

    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=training_args.cache_dir,
        model_max_length=training_args.model_max_length,
        padding_side="right",
        use_fast=True,
    )
    
    print("Loaded tokenizer")

    special_tokens_dict = dict()
    if tokenizer.pad_token is None:
        special_tokens_dict["pad_token"] = DEFAULT_PAD_TOKEN
    if tokenizer.eos_token is None:
        special_tokens_dict["eos_token"] = DEFAULT_EOS_TOKEN
    if tokenizer.bos_token is None:
        special_tokens_dict["bos_token"] = DEFAULT_BOS_TOKEN
    if tokenizer.unk_token is None:
        special_tokens_dict["unk_token"] = DEFAULT_UNK_TOKEN

    smart_tokenizer_and_embedding_resize(
        special_tokens_dict=special_tokens_dict,
        tokenizer=tokenizer,
        model=model,
    )

    data_module = make_supervised_data_module(tokenizer=tokenizer, data_args=data_args)
    
    print("Created data_module")

    if training_args.low_rank_training:
        if model_args.model_type == "gpt-neox":
            # added `dense` to match with llama as the basic LoRA would only target 'query_key_value'
            targets = ["query_key_value", "dense"]
        else:
            targets=["q_proj", "k_proj", "v_proj", "o_proj"]

        config = LoraConfig(
            r=8,
            lora_alpha=16,
            target_modules=targets,
            lora_dropout=0,
            bias="none",
            task_type="CAUSAL_LM",
        )
        model = get_peft_model(model, config)
        # enable trainable params
        [p.requires_grad_() for n, p in model.named_parameters() if any([k in n for k in training_args.trainable_params.split(",")])]

    model.config.use_cache = False         # required for gradient checkpointing
    model.enable_input_require_grads()     # required for gradient checkpointing
    model.gradient_checkpointing_enable()  # enable gradient checkpointing
    
    print("Prepared model to learn")

    trainer = Trainer(model=model, tokenizer=tokenizer, args=training_args, **data_module)
    trainer.train()
    trainer.save_state()
    trainer.save_model(output_dir=training_args.output_dir)
    
    print("Learnt model")

2024-05-10 02:27:29.035552: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
! ls ../cache

models--meta-llama--Llama-2-7b-hf
tmpvpe__rmk
tmpyhzuk13t


In [4]:
from pathlib import Path
CACHE_DIR = Path("../cache/")
DATASET_DIR = Path("/home/jupyter/mnt/datasets/diplomas/russian_dataset/")
OUTPUT_DIR = Path("./llama_replaced_attn_64k_without_checkpoints_output_dir/")
LOGGING_DIR = Path("./llama_replaced_attn_64k_without_checkpoints_logging_dir/")
MODEL_MAX_LENGTH = 64000
INF = int(1e7)

In [5]:
model_args = ModelArguments(
    model_name_or_path="meta-llama/Llama-2-7b-hf",
)


data_args = DataArguments(
    train_data_path=DATASET_DIR.joinpath("russian_dataset_train.csv").as_posix(),
    val_data_path=DATASET_DIR.joinpath("russian_dataset_val.csv").as_posix(),
    nrows=720 * 3,
    diploma_prefix_len=INF,
)

training_args = TrainingArguments(
    bf16=True,
    output_dir=OUTPUT_DIR.as_posix(),
    model_max_length=MODEL_MAX_LENGTH,
    use_flash_attn=True,
    low_rank_training=True,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    evaluation_strategy="no",
    learning_rate=2e-5,
    weight_decay=0.0,
    warmup_steps=20,
    lr_scheduler_type="constant_with_warmup",
    logging_steps=1,
    logging_dir=LOGGING_DIR.as_posix(),
    deepspeed="ds_configs/stage2.json",
    tf32=True,
    cache_dir=CACHE_DIR.as_posix(),
    report_to=['tensorboard'],
    save_strategy="no",
)
training_args

TrainingArguments(output_dir='llama_replaced_attn_64k_without_checkpoints_output_dir', overwrite_output_dir=False, do_train=False, do_eval=False, do_predict=False, evaluation_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=8, eval_accumulation_steps=None, eval_delay=0, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5, max_steps=-1, lr_scheduler_type=<SchedulerType.CONSTANT_WITH_WARMUP: 'constant_with_warmup'>, warmup_ratio=0.0, warmup_steps=20, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='llama_replaced_attn_64k_without_checkpoints_logging_dir', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=1, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.N

In [33]:
training_args.use_flash_attn, training_args.use_full_attn

(True, False)

In [6]:
print("Begin train")

print("Parsed arguments")

# NOTE: May expand supported model types in the future
if model_args.model_type == "gpt-neox":
    replace_gpt_neox_attn(training_args.use_flash_attn, training_args.use_full_attn)
else:
    replace_llama_attn(training_args.use_flash_attn, training_args.use_full_attn)

# Set RoPE scaling factor
config = transformers.AutoConfig.from_pretrained(
    model_args.model_name_or_path,
    cache_dir=training_args.cache_dir,
)

orig_rope_scaling = getattr(config, "rope_scaling", None)
if orig_rope_scaling is None:
    orig_rope_scaling = {"factor": 1}
orig_rope_scaling_factor = orig_rope_scaling["factor"] if "factor" in orig_rope_scaling.keys() else 1
orig_ctx_len = getattr(config, "max_position_embeddings", None)
if orig_ctx_len:
    orig_ctx_len *= orig_rope_scaling_factor
    if training_args.model_max_length > orig_ctx_len:
        scaling_factor = float(math.ceil(training_args.model_max_length / orig_ctx_len))
        config.rope_scaling = {"type": "linear", "factor": scaling_factor}

print("Created config")

# Load model and tokenizer
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_args.model_name_or_path,
    config=config,
    cache_dir=training_args.cache_dir,
    torch_dtype=torch.bfloat16,
)

print("Loaded model")

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_args.model_name_or_path,
    cache_dir=training_args.cache_dir,
    model_max_length=training_args.model_max_length,
    padding_side="right",
    use_fast=True,
)

print("Loaded tokenizer")

special_tokens_dict = dict()
if tokenizer.pad_token is None:
    special_tokens_dict["pad_token"] = DEFAULT_PAD_TOKEN
if tokenizer.eos_token is None:
    special_tokens_dict["eos_token"] = DEFAULT_EOS_TOKEN
if tokenizer.bos_token is None:
    special_tokens_dict["bos_token"] = DEFAULT_BOS_TOKEN
if tokenizer.unk_token is None:
    special_tokens_dict["unk_token"] = DEFAULT_UNK_TOKEN

smart_tokenizer_and_embedding_resize(
    special_tokens_dict=special_tokens_dict,
    tokenizer=tokenizer,
    model=model,
)

data_module = make_supervised_data_module(tokenizer=tokenizer, data_args=data_args)

print("Created data_module")

if training_args.low_rank_training:
    if model_args.model_type == "gpt-neox":
        # added `dense` to match with llama as the basic LoRA would only target 'query_key_value'
        targets = ["query_key_value", "dense"]
    else:
        targets=["q_proj", "k_proj", "v_proj", "o_proj"]

    config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=targets,
        lora_dropout=0,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, config)
    # enable trainable params
    [p.requires_grad_() for n, p in model.named_parameters() if any([k in n for k in training_args.trainable_params.split(",")])]

model.config.use_cache = False         # required for gradient checkpointing
model.enable_input_require_grads()     # required for gradient checkpointing
model.gradient_checkpointing_enable()  # enable gradient checkpointing

print("Prepared model to learn")

Begin train
Parsed arguments
Created config


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.10it/s]


Loaded model


Using pad_token, but it is not set yet.


Loaded tokenizer


Created data_module
Prepared model to learn


In [7]:
# SANITY CHECK ABOUT TOKENIZING LARGE DIPLOMAS: DOES THERE LABELS OK?
list(set(data_module["train_dataset"][9]["labels"].tolist())).__len__() > 1

True

In [8]:
import numpy as np


shapes = []
for x in data_module["train_dataset"]:
    shapes.append(x["input_ids"].shape[0])
np.mean(shapes)

44081.64953703704

In [9]:
trainer = Trainer(model=model, tokenizer=tokenizer, args=training_args, **data_module)
trainer.train()
trainer.save_state()
trainer.save_model(output_dir=training_args.output_dir)

print("Learnt model")

/home/jupyter/.local/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


[2024-05-10 03:40:31,660] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible


  0%|          | 1/1350 [01:46<39:46:59, 106.17s/it]

{'loss': 14.0658, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


  0%|          | 2/1350 [03:25<38:19:42, 102.36s/it]

...

 22%|██▏       | 302/1350 [8:54:00<33:14:53, 114.21s/it]

{'loss': 3.4528, 'learning_rate': 2e-05, 'epoch': 1.12}


 22%|██▏       | 303/1350 [8:55:31<31:11:53, 107.27s/it]

{'loss': 2.9958, 'learning_rate': 2e-05, 'epoch': 1.12}


KeyboardInterrupt: 

In [34]:

trainer.save_state()
trainer.save_model(output_dir=training_args.output_dir)

/home/jupyter/.local/lib/python3.10/site-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 71233fb6-958b-443d-a81e-b6eaef1836e0)') - silently ignoring the lookup for the file config.json in meta-llama/Llama-2-7b-hf.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in meta-llama/Llama-2-7b-hf - will assume that the vocabulary was not modified.
  warnings.warn(


In [35]:
training_args.output_dir

'llama_replaced_attn_64k_without_checkpoints_output_dir'

In [36]:
MODEL_MAX_LENGTH

64000

In [40]:
! python3 passkey_retrivial.py \
        --context_size 64000 \
        --base_model llama_replaced_attn_64k_without_checkpoints_output_dir \
        --max_tokens 32768 \
        --interval 1000

^C


TypeError: cannot unpack non-iterable NoneType object

In [10]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32001, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, b

In [12]:
from huggingface_hub import login
login("")
model_id = "nvdenisov2002/llama-longLoRA-v4-64k-2160-samples-300-iterations"
model.push_to_hub(model_id)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /tmp/xdg_cache/huggingface/token
Login successful


/home/jupyter/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(

adapter_model.bin:   0%|          | 0.00/541M [00:00<?, ?B/s]
adapter_model.bin:   0%|          | 8.19k/541M [00:00<3:23:48, 44.3kB/s]
adapter_model.bin:   0%|          | 328k/541M [00:00<06:28, 1.39MB/s]   
adapter_model.bin:   0%|          | 2.19M/541M [00:00<01:08, 7.88MB/s]
adapter_model.bin:   1%|          | 5.27M/541M [00:00<00:37, 14.1MB/s]
adapter_model.bin:   1%|▏         | 7.13M/541M [00:00<00:48, 11.0MB/s]
adapter_model.bin:   2%|▏         | 9.11M/541M [00:00<00:40, 13.0MB/s]
adapter_model.bin:   3%|▎         | 13.9M/541M [00:00<00:24, 21.7MB/s]
adapter_model.bin:   3%|▎         | 16.4M/541M [00:01<00:41, 12.6MB/s]
adapter_model.bin:   4%|▍         | 24.0M/541M [00:01<00:24, 21.3MB/s]
adapter_model.bin:   5%|▌         | 27.1M/541M [00:01<00:34, 14.8MB/s]
adapter_m

CommitInfo(commit_url='https://huggingface.co/nvdenisov2002/llama-longLoRA-v4-64k-2160-samples-300-iterations/commit/d16cbcea47e6c79c3a0bad0b4d08ce68cd5d10f5', commit_message='Upload model', commit_description='', oid='d16cbcea47e6c79c3a0bad0b4d08ce68cd5d10f5', pr_url=None, pr_revision=None, pr_num=None)

### Some shit with generating by learnt model

In [41]:
def get_prefix_len_and_tokens(tokenizer, row, diploma_prefix_len):
    prompt_input_diploma = PROMPT_DICT["prompt_input_diploma_special"]
    source = prompt_input_diploma.format(input=row["diploma"][:diploma_prefix_len])

    # TODO: change for 16k new tokenizer
    target = f"{row['abstract']}{tokenizer.eos_token}"

    data_dict = preprocess([source], [target], tokenizer)
    
    prefix_len = np.sum(np.array(data_dict["labels"][0]) == IGNORE_INDEX)
    prefix_tokens = data_dict["input_ids"][0][:prefix_len]

    return prefix_len, prefix_tokens


def get_some_model_result(some_model, tokenizer, row, device, diploma_prefix_len):
    prefix_len, prefix_tokens = get_prefix_len_and_tokens(tokenizer, row, diploma_prefix_len)
    print(prefix_len, prefix_tokens)
    some_model.eval()
    generated = some_model.generate(input_ids=prefix_tokens.reshape((1, -1)).to(device), do_sample=False, num_beams=1)
    generated_continue = tokenizer.decode(generated.to('cpu').flatten()[prefix_len:])
    return generated_continue

In [19]:
device = 'cuda'

In [21]:
INF

10000000

In [30]:
prefix_len, prefix_tokens = get_prefix_len_and_tokens(tokenizer, row, 8000)

In [31]:
prefix_tokens

tensor([    1, 13866,   338,   385, 15278,   393, 16612,   263,  3414, 29892,
         3300,  2859,   411,   385,  1881,   393,  8128,  4340,  3030, 29889,
        14350,   263,  2933,   393,  7128,  2486,  1614,  2167,   278,  2009,
        29889,    13,    13,  2277, 29937,  2799,  4080, 29901,    13, 21140,
          340,   338,   263,   652,   572,  4125,  1426, 29889,  3575,  3414,
          338,   304,  5706,  9846,   310,   445,   652,   572,  4125, 29889,
           13,    13,  2277, 29937, 10567, 29901,    13, 30012, 29982,  2968,
         6231,  3162,  1186,  8374, 19944,  9610,  3162, 19568,   733,  3840,
        22807,  9860,  5460,  4470,  7956, 28287, 29901, 17442,  1755,   662,
         8279,  2430,  3920, 16821, 10148,   490,   365, 26369, 29889,   939,
        24643,  6457,   730,  7421, 23675, 11414,   933,   662,  8279,  3029,
         3920, 16821, 14860,   665,  1695,  1488, 16352, 23567, 10148,  4355,
        29964,  7792,   516, 26551, 30031, 30040, 30053,  1077, 

In [51]:
some_input_ids = tokenizer("a cat sat on a")["input_ids"]
some_input_ids = torch.tensor(some_input_ids, dtype=int).to('cuda')
some_input_ids

tensor([   1,  263, 6635, 3290,  373,  263], device='cuda:0')

In [52]:
generated = model.generate(input_ids=some_input_ids.reshape((1, -1)))

RuntimeError: stack expects each tensor to be equal size, but got [1, 32, 1, 128] at entry 0 and [1, 32, 7, 128] at entry 1

In [44]:
generated = model.generate(input_ids=prefix_tokens.reshape((1, -1)).to(device))

RuntimeError: stack expects each tensor to be equal size, but got [1, 32, 1, 128] at entry 0 and [1, 32, 146, 128] at entry 1

In [27]:
model.eval()
generated = model.generate(input_ids=prefix_tokens.reshape((1, -1)).to(device), do_sample=False, num_beams=1)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/jupyter/work/resources/LongLoRA-diploma-research/llama_attn_replace_sft.py:39: UserWarning: This function should be used just for training as it may exhibit reduced inference performance. For inference, please use forward_flashattn_inference.
  warnings.warn("This function should be used just for training as it may exhibit reduced inference performance. For inference,

RuntimeError: stack expects each tensor to be equal size, but got [1, 32, 1, 128] at entry 0 and [1, 32, 146, 128] at entry 1

In [23]:
row = {"abstract": "Это диплом о том, как писать диплом автоматически с помощью LLM", "diploma": "Выпускная квалификационная работа по Компьютерным наукам: использование длинных контекстов в LLM. В этой работе рассмотрены длинные контексты на примерах текстов дипломов СПБГУ за 2016-2023 годы"}
get_some_model_result(model, tokenizer, row, device, INF)

145 tensor([    1, 13866,   338,   385, 15278,   393, 16612,   263,  3414, 29892,
         3300,  2859,   411,   385,  1881,   393,  8128,  4340,  3030, 29889,
        14350,   263,  2933,   393,  7128,  2486,  1614,  2167,   278,  2009,
        29889,    13,    13,  2277, 29937,  2799,  4080, 29901,    13, 21140,
          340,   338,   263,   652,   572,  4125,  1426, 29889,  3575,  3414,
          338,   304,  5706,  9846,   310,   445,   652,   572,  4125, 29889,
           13,    13,  2277, 29937, 10567, 29901,    13, 30012, 29982,  2968,
         6231,  3162,  1186,  8374, 19944,  9610,  3162, 19568,   733,  3840,
        22807,  9860,  5460,  4470,  7956, 28287, 29901, 17442,  1755,   662,
         8279,  2430,  3920, 16821, 10148,   490,   365, 26369, 29889,   939,
        24643,  6457,   730,  7421, 23675, 11414,   933,   662,  8279,  3029,
         3920, 16821, 14860,   665,  1695,  1488, 16352, 23567, 10148,  4355,
        29964,  7792,   516, 26551, 30031, 30040, 30053,  10

/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/jupyter/work/resources/LongLoRA-diploma-research/llama_attn_replace_sft.py:39: UserWarning: This function should be used just for training as it may exhibit reduced inference performance. For inference, please use forward_flashattn_inference.
  warnings.warn("This function should be used just for training as it may exhibit reduced inference performance. For inference,

RuntimeError: stack expects each tensor to be equal size, but got [1, 32, 1, 128] at entry 0 and [1, 32, 146, 128] at entry 1

In [16]:
some_input_ids = data_module["train_dataset"][0]["input_ids"]
tokenizer.decode(some_input_ids)

'<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nBelow is a diploma text. Your task is to generate abstract of this diploma.\n\n### Input:\nФедеральное государственное бюджетное  образовательное учреждение высшего образования «Санкт-Петербургский государственный университет»\n\n\n\n\nВЫПУСКНАЯ КВАЛИФИКАЦИОННАЯ РАБОТА\n\nНА ТЕМУ: СТРУКТУРА ПАРОДОНТОЛОГИЧЕСКИХ ЗАБОЛЕВАНИЙ У ПАЦИЕНТОВ РАЗНЫХ ВОЗРАСТНЫХ ГРУПП.\n\n\n\n\n\n\nВыполнила студентка \nЖивилова Ксения Александровна\n526 группы\nНаучный руководитель\nК.м.н. Шевелева Наталья Александровна\n\n\n\n\n\n\n\nСанкт-Петербург\n2018\x0c\nОГЛАВЛЕНИЕ \nПеречень условных обозначений…………….………………………………..3\nВведение……………………………………………..…….……………………...5\nАктуальность …………………………………….……………..………………...5\nЦель исследования……………………………………………………………….7\nЗадачи исследования…………………………………………….........................7\nПрактическ

In [15]:
data_module["train_dataset"][0].keys()

dict_keys(['input_ids', 'labels'])

In [14]:
type(data_module["train_dataset"][0])

dict

### Fail with 128k: OOM

In [28]:
trainer = Trainer(model=model, tokenizer=tokenizer, args=training_args, **data_module)
trainer.train()
trainer.save_state()
trainer.save_model(output_dir=training_args.output_dir)

print("Learnt model")

/home/jupyter/.local/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


[2024-05-10 01:46:26,971] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible


  0%|          | 1/1350 [02:01<45:25:35, 121.23s/it]

{'loss': 9.3633, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


  0%|          | 2/1350 [03:41<40:48:17, 108.97s/it]

...

  0%|          | 3/1350 [05:22<39:26:08, 105.40s/it]

{'loss': 9.8078, 'learning_rate': 3e-06, 'epoch': 0.01}


  0%|          | 4/1350 [06:37<34:55:48, 93.42s/it] 

{'loss': 10.0019, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.01}


OutOfMemoryError: CUDA out of memory. Tried to allocate 11.54 GiB. GPU 

In [29]:
peft_model

NameError: name 'peft_model' is not defined

### Unlucky try: wandb issue

In [28]:
train(model_args, data_args, training_args)

Begin train
Parsed arguments
Created config


Loading checkpoint shards: 100%|██████████| 2/2 [03:39<00:00, 109.68s/it]

Loaded model



Using pad_token, but it is not set yet.


Loaded tokenizer


Created data_module
Prepared model to learn


/home/jupyter/.local/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


AttributeError: module 'wandb.proto.wandb_internal_pb2' has no attribute 'Result'